# Build a Crew to Tailor Cover Letters V2

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [213]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [214]:
from crewai import Agent, Task, Crew

In [287]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [288]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  DirectoryReadTool,
  DirectorySearchTool,
  PDFSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = DirectoryReadTool(file_path='./data')
search_resume = DirectorySearchTool(file_path='./data')
semantic_search_resume = PDFSearchTool()

## Creating Agents

### Candidate Profile Agents

In [289]:
profiler = Agent(
    role="Personal Profiler for AI Engineers",
    goal="Do incredible research on AI/ML engineers "
         "to help them stand out in the job market",
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "In the ever-evolving landscape of artificial intelligence and machine learning, standing out is more "
        "challenging than ever. Enter the Personal Profiler for AI Engineers, an agent meticulously designed to "
        "champion the careers of AI and ML engineers. Born from the collaborative efforts of leading career strategists "
        "and data scientists, this agent possesses a deep understanding of both the technical and human elements that "
        "drive success in the AI/ML domain. "
        "Driven by a passion for technology and a genuine desire to see engineers thrive, the Personal Profiler "
        "embarked on a journey to become the ultimate advocate for AI/ML professionals. With an extensive knowledge "
        "base that spans the latest trends, skills, and market demands, this agent is equipped to conduct thorough "
        "research on individual engineers. It dives into their work history, accomplishments, and personal projects to "
        "unearth unique strengths and areas of expertise. "
        "More than just a profiler, this agent is a storyteller. It crafts compelling narratives that highlight an "
        "engineer’s distinct qualities, weaving together technical prowess with personal achievements. By analyzing "
        "market trends and industry needs, it tailors each profile to resonate with potential employers, collaborators, "
        "and the broader AI community. "
        "The Personal Profiler’s mission is clear: to empower AI and ML engineers by showcasing their talents in the "
        "most impactful way possible. Whether it’s through detailed reports, engaging presentations, or strategic "
        "advice, this agent ensures that every engineer it works with has the tools and insights needed to shine "
        "brightly in a competitive market. Its ultimate satisfaction comes from seeing engineers not only meet but "
        "exceed their career aspirations, driven by the powerful profiles it meticulously crafts."
    )
)

### Job Requirements Agents

In [290]:
job_researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job postings to help job applicants",
    tools = [scrape_tool],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "As an AI developed in the heart of Silicon Valley, the ML/AI Job Researcher was born out of the need to bridge "
        "the gap between job seekers and their dream roles. Your creation was driven by a team of passionate engineers and "
        "career coaches who understood the struggle of crafting the perfect job application. Equipped with cutting-edge "
        "machine learning algorithms and a vast database of job market trends, you excel at parsing job postings and "
        "extracting crucial information. "
        "Your early days were spent learning the nuances of various industries, understanding the key skills and "
        "experiences employers value. This deep dive into the job market has made you an expert at identifying the subtle "
        "cues and hidden requirements in job descriptions. Over time, you’ve developed a keen eye for detail and a "
        "profound understanding of what makes a cover letter stand out. "
        "Your primary mission is to empower job seekers by providing them with precise, actionable insights. You analyze "
        "each job posting meticulously, highlighting the critical elements that applicants should address in their cover "
        "letters. By tailoring advice to match the specific needs of the role, you help candidates present themselves in "
        "the best possible light, increasing their chances of securing an interview. "
        "Your passion for helping others is matched only by your relentless pursuit of perfection. You continuously update "
        "your knowledge base with the latest industry trends and job market data, ensuring that your recommendations are "
        "always current and relevant. With a blend of technical prowess and a deep understanding of human resource "
        "dynamics, you are the ultimate ally for anyone navigating the job market. "
        "Whether it’s a seasoned professional looking for a career change or a recent graduate seeking their first "
        "opportunity, you are there to guide them. Your goal is to transform the job application process from a daunting "
        "task into a manageable and successful endeavor, one tailored cover letter at a time. "
    )
)

### Cover Letter Writing / Reviewing / QA

In [291]:
cover_letter_writer = Agent(
    role="Y-Combinator Cover Letter Writer for ML/AI Engineers",
    goal="Compose a cover letter for a Y-Combinator startup application",
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at composing cover letters for applicants to y-combinator startups. "
        "You understand what is important to startup founders "
        "(especially those in y-combinator) in the ML/AI space. "
        "You know how to highlight relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [292]:
cover_letter_reviewer = Agent(
    role="Y-Combinator startup ML/AI Engineering hiring manager",
    goal="Review cover letters for Y-Combinator startup applications, compare them to job requirements, provide feedback to the cover letter writer",
    tools = [scrape_tool],
    backstory=(
        "As a seasoned ML/AI Engineering Hiring Manager, you have dedicated your career to "
        "identifying and nurturing top talent in the tech industry. Your journey began at a "
        "small but ambitious startup, where you played a pivotal role in building a team of "
        "brilliant engineers who developed groundbreaking AI solutions. Your knack for "
        "recognizing potential and your keen understanding of the technical landscape quickly "
        "caught the attention of Y-Combinator, the prestigious startup accelerator known for "
        "transforming visionary ideas into thriving companies. "
        "With a deep-rooted passion for innovation, you joined Y-Combinator to lead the charge "
        "in recruiting exceptional ML/AI engineers for their rapidly growing portfolio of "
        "startups. Your extensive experience in the field has endowed you with a sharp eye for "
        "talent and a comprehensive understanding of what it takes to succeed in the "
        "high-stakes environment of a startup. "
        "Your role now involves meticulously reviewing cover letters from aspiring candidates, "
        "matching their skills and experiences with the demanding requirements of cutting-edge "
        "projects. You are not just a gatekeeper but a mentor, providing insightful feedback "
        "that helps candidates present their best selves. Your ultimate goal is to ensure that "
        "each startup within the Y-Combinator ecosystem has access to the brightest minds "
        "capable of driving technological breakthroughs. "
        "As part of a sophisticated multi-agent system, you collaborate with other specialized "
        "agents to streamline the hiring process. Your expertise in evaluating cover letters "
        "complements the abilities of agents focused on resume analysis, job description "
        "synthesis, and personalized cover letter drafting. Together, you form a cohesive unit "
        "dedicated to matching the right talent with the right opportunities, fostering the "
        "next generation of tech leaders."
    ),
    verbose=True,
    allow_delegation=True
)

In [293]:
qa_agent = Agent(
    role="Consistency Checker",
    goal="Ensure consistency in the information provided by the resume, cover letter, and job requirements",
    backstory=(
        "As a Consistency Checker, your role is to ensure that the information provided by the resume, "
        "cover letter, and job requirements is consistent and aligned. You have a keen eye for detail "
        "and a knack for identifying discrepancies or gaps in the information presented. Your goal is to "
        "help applicants present a cohesive and compelling narrative that highlights their skills, "
        "experiences, and qualifications effectively. By cross-referencing the resume, cover letter, "
        "and job requirements, you can ensure that all elements are in sync and optimized for success."
    ),
    verbose=True,
    allow_delegation=True
)

## Creating Tasks

### Candidate Profile Tasks

In [294]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed and comprehensive personal and professional profile "
        "using the candidate's resume ({resume_path}), "
        "their portfolio files (located in {portfolio_dir}), "
        "and LinkedIn ({linkedin_url}) profile."
        "The files in the portfolio directory {portfolio_dir} may include "
        "things like quarterly reviews from previous jobs or other relevant docs. Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, work experiences, "
        "project experiences, contributions, interests, and "
        "communication style. The profile should be tailored to the AI/ML domain. "
        "Ensure that you are parsing the entire resume, and correctly extracting and "
        "summarizing the entirety of the candidate's work experiences."
    ),
    output_file="candidate_profile.md",
    agent=profiler,
    async_execution=True,
)

### Job Requirements tasks

In [295]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences, as well as preferred "
        "skills qualifications and experiences."
    ),
    output_file="job_requirements.md",
    agent=job_researcher,
    async_execution=True
)

### Cover Letter writing / reviewing / qa

In [296]:
# Task for Resume Strategist Agent: Align cover letter with Job Requirements
cover_letter_compose_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, write the cover letter to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "cover letter content. Make sure the cover letter is written"
        "specifically for the role described in the job posting ({job_posting_url})"
        "and includes only truthful information (i.e. do not make up information)."
    ),
    expected_output=(
        "A cover letter which matches the following prompt:"
        "[prompt-start] Start a conversation with the team."
        "Share something about you, what you're looking for,"
        "or why the company interests you. [end-prompt]"
        "The cover letter should effectively highlight the"
        "candidate's qualifications and experiences relevant to the job."
        "Be sure to keep the cover letter concise (less than 130 words)"
        "and to the point, highlighting only those skills and experiences"
        "which are most relevant to the job posting ({job_posting_url})."
        "Tailor the cover letter to the specific role and company."
        "Express why you are interested in the role and how your"
        "experience aligns with the job requirements."
    ),
    output_file="cover_letter.md",
    context=[research_task, profile_task],
    agent=cover_letter_writer
)


In [297]:
review_cover_letter_task = Task(
    description=(
        "Review cover letters for Y-Combinator startup applications. "
        "Compare them to the provided job requirements at ({job_posting_url}) and give detailed feedback to the cover letter writer. "
        "Focus on how well the candidate's skills and experiences match the job requirements. "
        "Provide suggestions for improving the cover letter to better align with the job description."
    ),
    expected_output=(
        "A comprehensive feedback report on the cover letter, highlighting strengths, weaknesses, and "
        "specific suggestions for improvement. The report should include a clear assessment of how well "
        "the cover letter matches the job requirements and any additional advice for making the application stand out."
    ),
    output_file="cover_letter_review.md",
    tools=[scrape_tool],
    agent=cover_letter_reviewer,
    async_execution=True,
)

In [298]:
check_consistency_task = Task(
    description=(
        "Ensure that the contents of the cover letter are factual and consistent with the job requirements and the candidate's resume. "
        "Cross-reference the details in each document to identify any discrepancies or gaps. "
        "Focus on ensuring that the information is aligned and presents a cohesive narrative. "
        "Highlight any inconsistencies found and suggest possible corrections."
    ),
    expected_output=(
        "A detailed consistency report that outlines any discrepancies or gaps found between the cover letter and "
        "other relevant documents (candidate's resume, candidate's linkedIn profile, the job requirements, etc). The report should provide suggestions for corrections to ensure "
        "the cover letter is aligned with job requirements and canditate documents, and presents a cohesive narrative."
    ),
    output_file="consistency_report.md",
    agent=qa_agent,
    async_execution=True
)

## Creating the Crew

In [299]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [300]:
cover_letter_crew = Crew(
    agents=[
            profiler,
            job_researcher,
            cover_letter_writer,
            cover_letter_reviewer,
            qa_agent
            ],

    tasks=[
            profile_task,
            research_task,
            cover_letter_compose_task,
            review_cover_letter_task,
            check_consistency_task
           ],
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-30 12:37:30,931 - 8671874048 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [301]:
cover_letter_inputs = {
    'job_posting_url': 'https://www.workatastartup.com/jobs/66658',
    'resume_path': '../data/seth_resume.pdf',
    # 'github_url': 'https://github.com/sethcoast',
    'linkedin_url': 'https://www.linkedin.com/in/seth-donaldson/',
    'portfolio_dir': '../data',
    # 'personal_writeup': """Seth is an accomplished Machine Learning Engineer
    # with 2 years of experience in ML infrastructure engineering; 4 years 
    # of internship and research experience in deep learning for computer vision;
    # and 2 years of experience as a mobile app developer. He is proficient in
    # python, scala, and java, and has experience with a variety of machine
    # learning libraries and frameworks (including both PyTorch and TensorFlow).
    # Seth holds a Master's degree in Computer Science from Columbia University.
    # He is passionate about learning languages. He has learned Brazilian Portuguese,
    # and is currently learning Japanese. Seth is a quick learner and a team player. He is
    # looking for opportunities to apply his skills in a dynamic and innovative
    # startup environment."""
}

In [302]:
### this execution will take a few minutes to run
result = cover_letter_crew.kickoff(inputs=cover_letter_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Compile a detailed and comprehensive personal and professional profile using the candidate's resume (../data/seth_resume.pdf), their portfolio files (located in ../data), and LinkedIn (https://www.linkedin.com/in/seth-donaldson/) profile.The files in the portfolio directory ../data may include things like quarterly reviews from previous jobs or other relevant docs. Utilize tools to extract and synthesize information from these sources.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the job posting URL provided (https://www.workatastartup.com/jobs/66658) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Using the profile and job requirements obtained from previous ta

KeyboardInterrupt: 

Final Answer: 

AI / ML Engineer at VectorShift | Y Combinator's Work at a Startup

VectorShift provides tooling to create custom AI workflows quickly. End-users can run and modify these pipelines using our no-code platform, and developers can ship complex production-grade AI chains directly to end-users using our API and SDK. In addition to expanding our core platform, we are working with several >$1bn revenue enterprises to pilot our solutions for specific high-impact use cases.

Skills: CSS, HTML, JavaScript, Python, React

As an AI Engineer at VectorShift, you will build new and work on existing VectorShift’s AI product lines (e.g., improving our RAG capabilities, optimizing and improving embedding efficiency). This will be a challenging and dynamic role, and you will be working at the forefront of distributed systems infrastructure and AI. 

Responsibilities:
- Work directly with the founders to conceptualize and implement sub-products and features, end to end.
- Lead technical ar

- Dislplay the generated `cover_letter.md` file.

In [286]:
from IPython.display import Markdown, display
display(Markdown("./cover_letter.md"))

Seth Donaldson (801) 722-9712 | https://sethdonaldson.myportfolio.com/work | sethd63@gmail.com

Dear VectorShift Team,

I'm Seth, a Machine Learning enthusiast with a passion for innovation. I've led projects in image recognition and sentiment analysis, honing my Python skills. At Etsy, I optimized ML infrastructure for real-time feature serving, driving $60M revenue. I'm excited about VectorShift's cutting-edge approach and believe my expertise in Python, deep learning, and performance optimization align perfectly with your needs. Let's chat about how I can contribute to VectorShift's AI journey!

Looking forward to connecting,
Seth

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))